In [265]:
import math
import copy
import random
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

In [266]:
# Define the blueprint of the city object (gene)
class city:

  def __init__(self, name, x, y):
    self._name = name
    self._x = x
    self._y = y
    self._visited = False

  @property
  def name(self):
    return self._name

  @name.setter
  def name(self, name):
    self._name = name


  @property
  def x(self):
    return self._x

  @x.setter
  def x(self, x):
    self._x = x


  @property
  def y(self):
    return self._y

  @y.setter
  def y(self, y):
    self._y = y


  @property
  def visited(self):
    return self._visited

  @visited.setter
  def visited(self, visited):
    self._visited = visited

In [267]:
# Define the blueprint of the chromosome object
class chromosome:

  def __init__(self, fitness, cost=0.0, genes=[]):
    self._genes = genes
    self._fitness = fitness
    self._cost = cost

  @property
  def fitness(self):
    return self._fitness


  @fitness.setter
  def fitness(self, fitness):
    self._fitness = fitness


  @property
  def cost(self):
    return self._cost


  @cost.setter
  def cost(self, cost):
    self._cost = cost


  @property
  def genes(self):
    return self._genes


  @genes.setter
  def genes(self, genes):
    self._genes = genes

## **1. Reading the Data**

In [268]:
df = pd.read_csv("/content/Data set CSV.csv")

In [269]:
df.head()

,City,x,y
0,1,5.500000e-08,9.860000e-09
1,2,-2.887330e+01,-7.980000e-08
2,3,-7.929160e+01,-2.140330e+01
3,4,-1.465770e+01,-4.338960e+01
4,5,-6.474730e+01,2.189820e+01


In [270]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   City    15 non-null     int64  
 1   x       15 non-null     float64
 2   y       15 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 492.0 bytes


## **2. Mapping the Data To Objects**

In [271]:
cities_objects = []     # Initialize an empty cities list

# Iterate through each row (city data) in the dataframe
for index, row in df.iterrows():
  x, y = row['x'], row['y']         # Get city coordinates
  name = row['City']                # Get city name
  city_obj = city(name, x, y)       # Create city object from the data
  cities_objects.append(city_obj)   # Append the city object to the cities

In [272]:
cities_objects

## **3. Get Euclidean Distance Between 2 Cities**

In [273]:
def get_euc_dist(city_1, city_2):
  '''
    Args:
      city_1 (city) :
      city_2 (city) :

    Returns:
      euc_dist (float) :
  '''

  euc_dist = np.sqrt((city_1.x - city_2.x)**2 + (city_1.y - city_2.y)**2)

  return euc_dist

## **4. Create The Distance Matrix**

In [274]:
def get_distance_matrix(cities_objects):
  '''
    Args:
      cities_objects (list) :

    Returns:
      distance_matrix (np.array) :
  '''

  n_cities = len(cities_objects)
  distance_matrix = np.zeros(shape=(n_cities, n_cities))

  for city_1 in cities_objects:

      for city_2 in cities_objects:

         distance_matrix[int(city_1.name)-1][int(city_2.name)-1] = get_euc_dist(city_1, city_2)


  return distance_matrix

In [275]:
distance_matrix = get_distance_matrix(cities_objects)

## **5. Generate the Initial Population**

In [276]:
# Genetic algorithm parameters:
population_size = 50
generations_count = 100
elitism_percentage = 0.02 # 2% of population (mean two chromosomes)
crossover_probability = 0.6
mutation_probability = 0.1

In [277]:
def get_cost(chosen_chromosome_genes):

  '''
    Args:
      chosen_chromosome_genes (list) : List of all city objects in the dataframe.

    Returns:
      cost (float): The total cost for the path.
  '''

  cost = 0

  # Get the distance between each 2 consecutive cities
  for idx, gene in enumerate(chosen_chromosome_genes[:-1]):
      curr_city, next_city = gene, chosen_chromosome_genes[idx+1]
      cost += distance_matrix[curr_city][next_city]

  # Get Distance Between The Last & First City
  first_city, last_city = chosen_chromosome_genes[0], chosen_chromosome_genes[-1]
  cost += distance_matrix[first_city][last_city]

  return cost

In [278]:
def generate_initial_population(population_size, cities):

  '''
    Args:
      population_size (int) : The desired size of the population.
      cities (list): List of all city objects in the dataframe.

    Returns:
      initial_population (list): The list containing initial population of chromosomes (routes, and genes represent cities).
  '''

  initial_population = []         # Initialize a list to store the initial population
  n_cities = len(cities)          # Get number of cities (unique genes)
  distance_matrix = get_distance_matrix(cities)       # Get the distance matrix for cities
  cities_sorted = sorted(cities, key=lambda city : city.name)     # Sort cities by their names (ascending order)

  # Generate the initial population
  for chromosome_idx in range(population_size):

    chosen_chromosome_genes = np.random.permutation(n_cities)     # Get random combination of indices for the genes (our chromosome)
    cost = get_cost(chosen_chromosome_genes)      # Get the cost for that chosen chromosome

    # Store Chromosome Data
    fitness = 1 / cost        # Get the fitness
    genes = [cities_sorted[idx] for idx in chosen_chromosome_genes]     # Get the genes of that chromosome (city objects)
    chromosome_obj = chromosome(fitness=fitness, cost=cost, genes=genes)      # Form the chromosome object

    # Add New Chromosome To our Population
    initial_population.append(chromosome_obj)


  return initial_population

In [279]:
initial_population = generate_initial_population(population_size, cities_objects)

In [280]:
print('Initial Population Size:', len(initial_population))

Initial Population Size: 50


## **6. Elitism**

In [281]:
def elitism(population, elitism_percentage):

  '''
    Args:
      population (list) :  The desired population to extract the elites from.
      elitism_percentage (float) : The desired percentage of elites.

    Returns:
      elited_chromosomes (list):  The elited chromosomes.
  '''

  population_size = len(population)     # The length of the population
  n_elited_chromosomes = math.ceil(population_size * elitism_percentage)      # Get number of elited chromosomes
  elited_chromosomes = sorted(population, key=lambda chromosome:chromosome.fitness, reverse=True)     # Sort the chromosomes by fitness (descending order)
  elited_chromosomes = elited_chromosomes[:n_elited_chromosomes]      # Get the elited chromosomes

  return elited_chromosomes

In [282]:
elited_chromosomes = elitism(initial_population, elitism_percentage)
print(elited_chromosomes)

## **7. Selection**

In [283]:
def k_tournment_selection(population, k=5):

  '''
    Args:
      population (list) : The desired population to extract the parent from.
      k (int) : The size of extracted sample from the population.

    Returns:
      selected_chromosome (chromosome):  The selected parent chromosome.
  '''

  population_size = len(population)       # Get the population size
  k_selected_chromosomes = []             # Initialize the list to store the samples

  # Extract the k samples randomly
  selected_indices = np.random.choice(population_size, k, replace=False)
  k_selected_chromosomes = [population[idx] for idx in selected_indices]

  k_selected_chromosomes = sorted(k_selected_chromosomes, key= lambda chrom : chrom.fitness, reverse=True)      # Sort the chosen samples by fitness (descending order)
  selected_chromosome = k_selected_chromosomes[0]         # Get the chromosome with highest fitness as a chosen parent

  return selected_chromosome

In [284]:
selected_chromosome = k_tournment_selection(initial_population, k=10)
print(selected_chromosome)

## **8. Crossover**

In [285]:
def partially_mapped_crossover(cities, population, elitism_percentage, crossover_probability):

  '''
    Args:
      population (list) : The desired population to perform crossover with & form the new generation.
      elitism_percentage (float) :  The desired elites percentage.
      crossover_probability (float) :  The crossover threshold, beyond it we will keep parents, below it we will form new children.

    Returns:
      generation_chromosomes (list):  The new generation list with preserved space for elites.
  '''

  n_population = len(population)    # Get the population size
  n_cities = len(population[0].genes)   # Get the number of unique cities (genes)


  n_elites = math.ceil(elitism_percentage * n_population)     # Get number of elites
  n_crossover = math.ceil((n_population - n_elites) / 2)      # Get the number by which we will perform crossover times
  generation_chromosomes = [0] * n_elites               # Initialize the generation list with placeholder for elites


  # Produce a genration using partially mapped crossover technique (n_crossover times)
  for crossover_time in range(n_crossover):

    random_probability = np.random.rand()     # Generate random probability to see if we will perform crossover or not
    parent_1 = k_tournment_selection(population, k=5)        # Get first parent
    parent_2 = k_tournment_selection(population, k=5)        # Get second parent


    # Perform Crossover
    if random_probability < crossover_probability:

      # Generate random cutoffs
      cutoff_1 = np.random.randint(0, n_cities)
      cutoff_2 = np.random.randint(0, n_cities)

      child_1 = copy.deepcopy(parent_1)       # Initialize the first child
      child_2 = copy.deepcopy(parent_2)       # Initialize the second child


      # Exchange the desired part
      if cutoff_1 > cutoff_2:
        cutoff_1, cutoff_2 = cutoff_2, cutoff_1

      for parent_idx in range(cutoff_1, cutoff_2 + 1):

        val_1 , val_2 = parent_1.genes[parent_idx], parent_2.genes[parent_idx]        # Get the items (cities) in the middle part for both parents item by item
        child_1_desired_idx, child_2_desired_idx = 0, 0

        # Search for the items (cities) to exchange
        for child_idx, zipped_genes in enumerate(zip(child_1.genes, child_2.genes)):

          gene_1, gene_2 = zipped_genes
          if gene_1.name == val_2.name:
            child_1_desired_idx = child_idx
            child_1.genes[child_1_desired_idx] = val_1


          if gene_2.name == val_1.name:
            child_2_desired_idx = child_idx
            child_2.genes[child_2_desired_idx] = val_2


        child_1.genes[parent_idx] = val_2
        child_2.genes[parent_idx] = val_1


      # Evaluate Child 1 cost & fitness
      child_1_genes_indices = [int(gene.name)-1 for gene in child_1.genes]
      child_1.cost = get_cost(child_1_genes_indices)
      child_1.fitness = 1 / child_1.cost

      # Evaluate Child 2 cost & fitness
      child_2_genes_indices = [int(gene.name)-1 for gene in child_2.genes]
      child_2.cost = get_cost(child_2_genes_indices)
      child_2.fitness = 1 / child_2.cost


      # Append the newly generated children
      generation_chromosomes.append(child_1)
      generation_chromosomes.append(child_2)

    # Leave Children as they are
    else:
      generation_chromosomes.append(parent_1)
      generation_chromosomes.append(parent_2)


    # Make sure that no overflow happens
    while (len(generation_chromosomes) > n_population):
      generation_chromosomes.pop()

  return generation_chromosomes

In [286]:
generation_chromosomes = partially_mapped_crossover(cities_objects, initial_population, elitism_percentage, crossover_probability)
print(len(generation_chromosomes), 'Generated Chromosomes:\n', generation_chromosomes)

50 Generated Chromosomes:
 [0, <__main__.chromosome object at 0x7bc198130390>, <__main__.chromosome object at 0x7bc1836a2650>, <__main__.chromosome object at 0x7bc18364cf50>, <__main__.chromosome object at 0x7bc183524c90>, <__main__.chromosome object at 0x7bc18352fbd0>, <__main__.chromosome object at 0x7bc183519dd0>, <__main__.chromosome object at 0x7bc19812cb90>, <__main__.chromosome object at 0x7bc183637bd0>, <__main__.chromosome object at 0x7bc1836a0c10>, <__main__.chromosome object at 0x7bc183518150>, <__main__.chromosome object at 0x7bc1836ea250>, <__main__.chromosome object at 0x7bc183708850>, <__main__.chromosome object at 0x7bc18351a290>, <__main__.chromosome object at 0x7bc183709dd0>, <__main__.chromosome object at 0x7bc18364cf50>, <__main__.chromosome object at 0x7bc183519dd0>, <__main__.chromosome object at 0x7bc1836353d0>, <__main__.chromosome object at 0x7bc183709cd0>, <__main__.chromosome object at 0x7bc1836eb050>, <__main__.chromosome object at 0x7bc1836f6c50>, <__main__

## **9. Mutation**

In [287]:
def swap_mutation(population, elitism_percentage, mutation_probability):

  '''
    Args:
      population (list) : The desired population to perform crossover with & form the new generation.
      elitism_percentage (float) :
      mutation_probability (float) : The mutation threshold, below this threshold mutate the chosen random chromosome, below it leave it as it is.

    Returns:
      None
  '''

  n_population = len(population)        # Get the size of the population
  n_cities = len(population[0].genes)     # Get the number of the unique cities (genes)
  n_elites = math.ceil(n_population * elitism_percentage)       # Get the number of elites to preserve


  # Iterate (n_population - n_elites) times and get a random chromosome each time
  for chrom in range(n_population - n_elites):

    random_parent_idx = np.random.randint(n_elites, n_population)   # Generate a random index
    parent_chromosome = population[random_parent_idx]               # Get the chromosome associated with that random index

    random_probability = np.random.rand()       # Generate a random probability


    # Perform swap mutation on that randomly chosen chromosome
    if (random_probability < mutation_probability):

      # Generate the random indices to be swapped
      random_idx_1 = np.random.randint(n_cities)
      random_idx_2 = np.random.randint(n_cities)

      # Create the new mutated object to be
      new_parent_chromosome = copy.deepcopy(parent_chromosome)

      # Perform Swapping
      new_parent_chromosome.genes[random_idx_1] = parent_chromosome.genes[random_idx_2]
      new_parent_chromosome.genes[random_idx_2] = parent_chromosome.genes[random_idx_1]

      # Evaluate New cost & fitness
      new_parent_genes_indices = [int(gene.name)-1 for gene in new_parent_chromosome.genes]
      new_parent_chromosome.cost = get_cost(new_parent_genes_indices)
      new_parent_chromosome.fitness = 1 / new_parent_chromosome.cost

      # Replace the parent chromosome with the mutated one
      population[random_parent_idx] = new_parent_chromosome

In [288]:
swap_mutation(initial_population, elitism_percentage, mutation_probability)

## **10. Genetic Algorithm**

In [291]:
def genetic_algorithm(cities_objects, population_size, generations_count, elitism_percentage, crossover_probability, mutation_probability):

  '''
    Args:
      cities_objects (list) : The list containing the city objects.
      population_size (int) : The desired size of the population.
      generations_count (int) : The desired number of generations we update (stopping criterion).
      elitism_percentage (float) :  The desired elites percentage.
      crossover_probability (float) :  The crossover threshold, beyond it we will keep parents, below it we will form new children.
      mutation_probability (float) :  The mutation threshold, below this threshold mutate the chosen random chromosome, below it leave it as it is.

    Returns:
      fittest_chrmosome (chromosome) :  The chromosome with the highest fitness in the final generation.
  '''

  # Get the initial population
  initial_population = generate_initial_population(population_size, cities_objects)


  # Print Initial Cost
  sorted_chrmosomes = sorted(initial_population, key=lambda chromosome : chromosome.fitness, reverse=True)   # Sort the chromosomes by fitness
  fittest_chrmosome = sorted_chrmosomes[0]    # Get the fittest chromosome
  print('Initial Cost: ', fittest_chrmosome.cost)


  # Get the unique city objects (genes)
  unique_genes = initial_population[0].genes

  # Initialize an empty list to store the new generation
  new_generation = []

  # Get the number of expected elites
  n_elites = math.ceil(elitism_percentage * population_size)

  # Create various generations (generations_count times)
  for generation in range(generations_count):

    elited_chromosomes = elitism(initial_population, elitism_percentage)      # Get the elited chromosomes
    new_generation = partially_mapped_crossover(cities_objects, initial_population, elitism_percentage, crossover_probability)        # Perform crossover to generate the new generation list
    new_generation[:n_elites] = elited_chromosomes              # Place the elited chromosomes in their respective place in the new generation list
    swap_mutation(new_generation, elitism_percentage, mutation_probability)         # Mutate the new generation list
    initial_population = new_generation             # Replace the initial population (parents) with the new generation


  sorted_chrmosomes = sorted(initial_population, key=lambda chromosome : chromosome.fitness, reverse=True)   # Sort the chromosomes by fitness
  fittest_chrmosome = sorted_chrmosomes[0]    # Get the fittest chromosome


  return fittest_chrmosome

In [290]:
fittest_chrmosome = genetic_algorithm(cities_objects, population_size, generations_count, elitism_percentage, crossover_probability, mutation_probability)
print('Final Cost: ', fittest_chrmosome.cost)     # Print Final Cost

Initial Cost:  423.7806277362588
Final Cost:  284.38103676026583
